In [1]:
#Import Library
import pandas as pd

In [2]:
#Import Dataset
data=pd.read_csv('database/data/external/Police_Use_of_Force.csv')

In [3]:
#show data columns
data.columns

Index(['X', 'Y', 'PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem',
       'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber',
       'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction',
       'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct',
       'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear',
       'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude',
       'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'],
      dtype='object')

In [4]:
#add year, month, day, hour into dataset
data['date']=data['ResponseDate'].apply(lambda x:x.split(' ')[0])
data['time']=data['ResponseDate'].apply(lambda x:x.split(' ')[1])
data['year']=data['date'].apply(lambda x:x.split('/')[0])
data['month']=data['date'].apply(lambda x:x.split('/')[1])
data['day']=data['date'].apply(lambda x:x.split('/')[2])
data['hour']=data['time'].apply(lambda x:x.split(':')[0])

In [5]:
#drop duplicates columns
data = data.drop(['X','Y','CenterX','CenterY','DateAdded'],axis=1)

In [6]:
#columns 
data.columns

Index(['PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem',
       'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber',
       'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction',
       'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct',
       'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear',
       'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude',
       'CenterLongitude', 'OBJECTID', 'date', 'time', 'year', 'month', 'day',
       'hour'],
      dtype='object')

In [7]:
data.head()

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,SubjectRole,SubjectRoleNumber,...,CenterGBSID,CenterLatitude,CenterLongitude,OBJECTID,date,time,year,month,day,hour
0,15273935,08-013527,2008/01/14 01:46:49+00,Suspicious Vehicle,No,DWIREF,NaN,1,A,1.0,...,25963,44.979153,-93.277322,1,2008/01/14,01:46:49+00,2008,01,14,01
1,15273936,08-025804,2008/01/27 02:31:10+00,Assault in Progress,No,ASLT4,Yes,2,A,1.0,...,17120,44.981655,-93.271963,2,2008/01/27,02:31:10+00,2008,01,27,02
2,15273937,08-032626,2008/02/02 22:40:31+00,Motor Vehicle Chase,No,FLEE,Yes,4,A,1.0,...,16331,44.984396,-93.304503,3,2008/02/02,22:40:31+00,2008,02,02,22
3,15273938,08-041372,2008/02/12 01:36:17+00,Assault in Progress,Yes,FORCE,No,1,OT,1.0,...,22871,45.021301,-93.244784,4,2008/02/12,01:36:17+00,2008,02,12,01
4,15273939,08-045117,2008/02/16 12:18:53+00,Domestic Abuse-In Progress,Yes,DASLT5,NaN,1,A,1.0,...,16878,44.953739,-93.253096,5,2008/02/16,12:18:53+00,2008,02,16,12


# subject_tb

In [8]:
subject_tb = data[['OBJECTID','Race','Sex','EventAge','SubjectInjury','SubjectRole','SubjectRoleNumber','TypeOfResistance']]
subject_tb = subject_tb.rename(columns={'OBJECTID':'subject_id','Race':'race','Sex':'sex','EventAge':'age',
                                     'SubjectInjury':'has_injury','SubjectRole':'role',
                                     'SubjectRoleNumber':'role_number','TypeOfResistance':'resistance'})
subject_tb = subject_tb.fillna('Null')
subject_tb.head()

,subject_id,race,sex,age,has_injury,role,role_number,resistance
0,1,Black,Male,28,Null,A,1,Fled in Vehicle
1,2,Black,Male,32,Yes,A,1,Assaulted Officer
2,3,White,Male,26,Yes,A,1,Fled in Vehicle
3,4,Black,Male,18,No,OT,1,Tensed
4,5,Black,Male,26,Null,A,1,Tensed


In [9]:
subject_tb.to_csv('database/data/external/subject_tb.csv')

# city_tb

In [10]:
city_tb = pd.DataFrame({'city_id':0,'city_name':'Minneapolis'},index=[0])
city_tb.head()

,city_id,city_name
0,0,Minneapolis


In [11]:
city_tb.to_csv('database/data/external/city_tb.csv')

# city_summary_tb

In [12]:
city_summary_tb = data[['year','TotalCityCallsForYear']]
city_summary_tb = city_summary_tb.drop_duplicates()
city_summary_tb = city_summary_tb.reset_index(drop=True)
city_summary_tb = city_summary_tb.reset_index()
city_summary_tb['city_id'] = 0
city_summary_tb = city_summary_tb[['index','city_id','year','TotalCityCallsForYear']]
city_summary_tb = city_summary_tb.rename(columns={'index':'city_summary_id','TotalCityCallsForYear':'total_calls'})
city_summary_tb.head()

,city_summary_id,city_id,year,total_calls
0,0,0,2008,322402
1,1,0,2009,316150
2,2,0,2010,314926
3,3,0,2011,313095
4,4,0,2012,324520


In [13]:
city_summary_tb.to_csv('database/data/external/city_summary_tb.csv')

# case_tb

In [14]:
case_tb=data[['OBJECTID','CaseNumber','Is911Call','Problem','PrimaryOffense','ResponseDate','CenterLatitude',
             'CenterLongitude','Precinct','Neighborhood']]
case_tb=case_tb.rename(columns={'OBJECTID':'case_id','CaseNumber':'case_number','Is911Call':'is_911_call',
                       'Problem':'problem','PrimaryOffense':'primary_offense','CenterLatitude':'latitude',
                       'CenterLongitude':'longitude','Precinct':'precinct_id','Neighborhood':'neighborhood_id'})

case_tb['date']=case_tb['ResponseDate'].apply(lambda x:x.split(' ')[0])
case_tb['time']=case_tb['ResponseDate'].apply(lambda x:x.split(' ')[1])
case_tb['year']=case_tb['date'].apply(lambda x:x.split('/')[0])
case_tb['month']=case_tb['date'].apply(lambda x:x.split('/')[1])
case_tb['day']=case_tb['date'].apply(lambda x:x.split('/')[2])
case_tb['hour']=case_tb['time'].apply(lambda x:x.split(':')[0])
case_tb['city_id']=1
case_tb=case_tb.drop('ResponseDate',axis=1)

case_tb.head()

,case_id,case_number,is_911_call,problem,primary_offense,latitude,longitude,precinct_id,neighborhood_id,date,time,year,month,day,hour,city_id
0,1,08-013527,No,Suspicious Vehicle,DWIREF,44.979153,-93.277322,01,Downtown West,2008/01/14,01:46:49+00,2008,01,14,01,1
1,2,08-025804,No,Assault in Progress,ASLT4,44.981655,-93.271963,01,Downtown West,2008/01/27,02:31:10+00,2008,01,27,02,1
2,3,08-032626,No,Motor Vehicle Chase,FLEE,44.984396,-93.304503,04,Near - North,2008/02/02,22:40:31+00,2008,02,02,22,1
3,4,08-041372,Yes,Assault in Progress,FORCE,45.021301,-93.244784,02,Audubon Park,2008/02/12,01:36:17+00,2008,02,12,01,1
4,5,08-045117,Yes,Domestic Abuse-In Progress,DASLT5,44.953739,-93.253096,03,Midtown Phillips,2008/02/16,12:18:53+00,2008,02,16,12,1


In [15]:
case_tb.to_csv('database/data/external/case_tb.csv')

# precinct_tb

In [16]:
precinct_list=list(data['Precinct'].unique())
precinct_list[-2]='Null'
precinct_dict={}
n=0
for i in (precinct_list):
    n=n+1
    precinct_dict.update({i:n})

In [17]:
precinct_tb = data['Precinct'].drop_duplicates().fillna("Null").reset_index(drop=True).reset_index()
precinct_tb['index'] = precinct_tb['Precinct'].apply(lambda x:precinct_dict[x])
precinct_tb = precinct_tb.rename(columns={'index':'precinct_id','Precinct':"precinct_name"})
precinct_tb.head()

,precinct_id,precinct_name
0,1,01
1,2,04
2,3,02
3,4,03
4,5,05


In [18]:
precinct_tb.to_csv('database/data/external/precinct_tb.csv')

# precinct_summary

In [19]:
precinct_raw = data['Precinct'].fillna("Null").reset_index(drop=True).reset_index()
precinct_raw['index'] = precinct_raw['Precinct'].apply(lambda x:precinct_dict[x])
precinct_raw =precinct_raw.rename(columns={'index':'precinct_id','Precinct':"precinct_name"})

In [20]:
precinct_summary_tb_b4=precinct_raw.join(data['year'])
precinct_summary_tb_b4['count']=1

In [21]:
total=(precinct_summary_tb_b4.groupby(['precinct_id','precinct_name','year']).sum())['count']
total_list = list(total)

In [22]:
precinct_summary_tb = precinct_summary_tb_b4.sort_values(['precinct_id','precinct_name','year']).drop_duplicates().reset_index(drop=True).reset_index()
precinct_summary_tb = precinct_summary_tb.rename(columns = {'index':'precinct_summary_id'})
precinct_summary_t = precinct_summary_tb.drop(['count'],axis=1)
precinct_summary_tb['total_calls'] = total_list
precinct_summary_tb.head()

,precinct_summary_id,precinct_id,precinct_name,year,count,total_calls
0,0,1,01,2008,1,1000
1,1,1,01,2009,1,956
2,2,1,01,2010,1,874
3,3,1,01,2011,1,972
4,4,1,01,2012,1,735


In [23]:
precinct_summary_tb.to_csv('database/data/external/precinct_summary_tb.csv')

# neightborhood_tb

In [24]:
neighborhood_list = list(data['Neighborhood'].fillna('Null').drop_duplicates())
neighborhood_dict={}
n=0
for i in neighborhood_list:
    n=n+1
    neighborhood_dict.update({i:n})

In [25]:
neighborhood_tb = pd.DataFrame({'neighborhood_id':list(neighborhood_dict.values()),
                                'neighborhood_name':list(neighborhood_dict.keys())})

In [26]:
neighborhood_tb.head()

,neighborhood_id,neighborhood_name
0,1,Downtown West
1,2,Near - North
2,3,Audubon Park
3,4,Midtown Phillips
4,5,Ventura Village


In [27]:
neighborhood_tb.to_csv('database/data/external/neighborhood_tb.csv')

# neightborhood_summary_tb

In [28]:
neighborhood_summary_raw = data[['Neighborhood','year']].copy()
neighborhood_summary_raw['count']=1
neighborhood_summary_raw['Neighborhood'].fillna('Null')
neighborhood_summary_raw['neighborhood_id']=neighborhood_summary_raw['Neighborhood'].fillna('Null')\
                                            .apply(lambda x:neighborhood_dict[x])
neighborhood_summary_raw = neighborhood_summary_raw.drop('Neighborhood',axis=1)

In [29]:
total=(neighborhood_summary_raw.groupby(['neighborhood_id','year']).sum())['count']
total_list = list(total)

In [30]:
neighborhood_summary_tb = neighborhood_summary_raw.copy()
neighborhood_summary_tb = neighborhood_summary_tb.drop_duplicates().sort_values(['neighborhood_id','year'])
neighborhood_summary_tb = neighborhood_summary_tb[['neighborhood_id','year']]
neighborhood_summary_tb['total_calls'] = total_list
neighborhood_summary_tb = neighborhood_summary_tb.reset_index(drop=True).reset_index()
neighborhood_summary_tb = neighborhood_summary_tb.rename(columns={'index':'neigborhood_summary_id'})
neighborhood_summary_tb.head()

,neigborhood_summary_id,neighborhood_id,year,total_calls
0,0,1,2008,807
1,1,1,2009,814
2,2,1,2010,720
3,3,1,2011,819
4,4,1,2012,598


In [31]:
neighborhood_summary_tb.to_csv('database/data/external/neighborhood_summary_tb.csv')